### 피봇테이블과 그룹분석

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
pd.__version__

'1.0.5'

#### 피봇테이블

In [3]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [4]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [5]:
df1.set_index(["도시", "연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

In [6]:
df1.pivot(["지역", "도시"], "연도", "인구")

ValueError: Length of passed values is 8, index implies 2.

#### 그룹분석

In [7]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [27]:
groups = df2.groupby(df2.key1)
groups

In [28]:
groups.groups

{'A': Int64Index([0, 1, 4], dtype='int64'),
 'B': Int64Index([2, 3], dtype='int64')}

In [10]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [11]:
df2.groupby(df2.key1).sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [12]:
df2.groupby(df2.key1).mean()

,data1,data2
key1,,
A,2.666667,26.666667
B,3.500000,35.000000


In [13]:
df2.groupby(df2['key1']).sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [14]:
df2.groupby(df2.key1)['data1'].sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [15]:
df2.groupby(df2.key1)[('data1', 'data2')].sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [16]:
df2.groupby(df2.key1)[['data1']].sum()

,data1
key1,
A,8
B,7


In [17]:
df2.groupby(df2.key1)[('data1',)].sum()

,data1
key1,
A,8
B,7


In [18]:
df2.data1.groupby([df2.key1, df2.key2]).sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [19]:
df2.groupby([df2.key1, df2.key2])['data1'].sum()

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [20]:
df2.groupby([df2.key1, df2.key2]).sum()['data1']

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [21]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [22]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [23]:
iris.groupby(iris.species).mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [24]:
iris.groupby(iris.species)['sepal_length'].describe()

,count,mean,std,min,25%,50%,75%,max
species,,,,,,,,
setosa,50.0,5.006,0.352490,4.3,4.800,5.0,5.2,5.8
versicolor,50.0,5.936,0.516171,4.9,5.600,5.9,6.3,7.0
virginica,50.0,6.588,0.635880,4.9,6.225,6.5,6.9,7.9


In [25]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [26]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)

iris["petal_length_class"] = iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


#### pivot_table

In [7]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [8]:
df1.pivot_table("인구", "도시", "연도")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [9]:
df1.pivot_table("인구", "도시", "연도", margins=True, margins_name="평균")

연도,2005,2010,2015,평균
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
평균,6637546.5,4429292.0,5414500.0,5.350809e+06


In [10]:
df1["인구"].mean()

5350808.625

In [11]:
df1.인구.mean()

5350808.625

In [15]:
df1.pivot_table("인구", index=["지역", "도시"], columns="연도")

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

##### tip dataset으로 분석

In [17]:
import seaborn as sns
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [18]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [19]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


In [20]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [21]:
# 성별 데이터 갯수
tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

In [22]:
# 성별과 흡연유무로 나누어 데이터의 갯수
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [23]:
# 성별과 흡연유무 피봇 테이블
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc="count", margins=True)

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


In [24]:
# 성별 평균 팁 비율
tips.groupby("sex")[["tip_pct"]].mean()

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [25]:
# 흡연 여부에 따른 평균 팁 비율
tips.groupby("smoker")[["tip_pct"]].mean()

,tip_pct
smoker,
Yes,0.163196
No,0.159328


In [26]:
# 성별과 흡연 여부에 따른 평균 팁 비율
tips.pivot_table("tip_pct", ["sex", "smoker"])

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

In [27]:
tips.pivot_table("tip_pct", "sex", "smoker")

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [28]:
# 요일별 평균 팁 비율
tips.groupby("day")[["tip_pct"]].mean()

,tip_pct
day,
Thur,0.161276
Fri,0.169913
Sat,0.153152
Sun,0.166897


In [29]:
# 시간별 평균 팁 비율
tips.groupby("time")[["tip_pct"]].mean()

,tip_pct
time,
Lunch,0.164128
Dinner,0.159518


In [30]:
tips.pivot_table('tip_pct', 'day', 'time', margins=True)

time,Lunch,Dinner,All
day,,,
Thur,0.161301,0.159744,0.161276
Fri,0.188765,0.158916,0.169913
Sat,NaN,0.153152,0.153152
Sun,NaN,0.166897,0.166897
All,0.164128,0.159518,0.160803


In [31]:
tips.pivot_table('tip_pct', index=['day', 'time'], columns='size', margins=True, margins_name='평균')

size                1         2         3         4         5         6  \
day  time                                                                 
Thur Lunch   0.181728  0.164024  0.144599  0.145515  0.121389  0.173706   
     Dinner       NaN  0.159744       NaN       NaN       NaN       NaN   
Fri  Lunch   0.223776  0.181969  0.187735       NaN       NaN       NaN   
     Dinner       NaN  0.162659       NaN  0.117750       NaN       NaN   
Sat  Dinner  0.231832  0.155289  0.151439  0.138289  0.106572       NaN   
Sun  Dinner       NaN  0.180870  0.152662  0.153168  0.159839  0.103799   
평균           0.217292  0.165719  0.152157  0.145949  0.141495  0.156229   

size               평균  
day  time              
Thur Lunch   0.161301  
     Dinner  0.159744  
Fri  Lunch   0.188765  
     Dinner  0.158916  
Sat  Dinner  0.153152  
Sun  Dinner  0.166897  
평균           0.160803

In [33]:
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

In [34]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].agg(peak_to_peak)

tip_pct
sex    smoker          
Male   Yes     0.674707
       No      0.220186
Female Yes     0.360233
       No      0.195876

In [35]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].agg(["mean", peak_to_peak])

tip_pct             
                   mean peak_to_peak
sex    smoker                       
Male   Yes     0.152771     0.674707
       No      0.160669     0.220186
Female Yes     0.182150     0.360233
       No      0.156921     0.195876

In [36]:
tips.groupby(["sex", "smoker"]).agg(
    {'tip_pct': 'mean', 'total_bill': peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

In [37]:
tips.pivot_table('size', ['time', 'sex', 'smoker'], 'day',
                 aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43